In [ ]:
import json
import os
from typing import List, Dict

import numpy as np
import torch
from matplotlib import pyplot as plt

from utils.losses import calculate_mape_loss

model_ids: List[str] = [
    # 'SimpleNeuralNet2021_12_02_11_51_54_168646',
    # 'SimpleNeuralNet2021_12_02_11_11_55_435483',
    # 'LinearRegression2021_12_02_12_25_39_613388',
    'TimeSeriesTransformer2021_12_02_16_57_45_077229',
    # 'TimeSeriesTransformerWithConvolutionalAttention2021_12_08_12_03_29_886202',
    # 'TimeSeriesTransformerWithConvolutionalAttention2021_12_15_11_57_08_935348',
    'LinearRegression2021_12_02_12_25_39_613388',
    # 'TimeSeriesTransformerWithConvolutionalAttention2021_12_15_12_36_39_801430',
    'TimeSeriesTransformerWithConvolutionalAttention2021_12_08_12_03_29_886202',
    'TimeSeriesTransformerWithConvolutionalAttention2021_12_15_12_36_39_801430'
]

comparison_model_id: str = 'SimpleNeuralNet2021_12_02_11_11_55_435483'

experiments = []
for model_id in model_ids:
    with open(os.path.join('..', 'experiments', 'archive', model_id + '.json')) as file:
        experiments.append(json.load(file))

with open(os.path.join('..', 'experiments', 'archive', comparison_model_id + '.json')) as file:
    comparison_experiment = json.load(file)

In [ ]:
def create_ensemble_model(exps):
    length = len(exps[0]['evaluation']['expected_predicted_comparison'])
    predicted = np.array([np.array([0 for _ in range(0, 24)], dtype=np.float32) for _ in range(0, length)],
                         dtype=np.float32)
    expected = np.array([np.array([0 for _ in range(0, 24)], dtype=np.float32) for _ in range(0, length)],
                        dtype=np.float32)
    print(predicted.shape)
    for experiment in exps:
        for index, d in enumerate(experiment['evaluation']['expected_predicted_comparison']):
            predicted[index] += np.array(d['predicted'])
            expected[index] = np.array(d['expected'])  # it is expected that the model are trained on the same data
    predicted /= len(exps)
    return predicted, expected


p, e = create_ensemble_model(experiments)
mape_loss = calculate_mape_loss(torch.Tensor(p), torch.Tensor(e)).item()
print(mape_loss)

In [ ]:
mape_losses_by_prediction_variable = dict()
for index in range(0, 24):
    t1 = torch.Tensor(e[:, index])
    t2 = torch.Tensor(p[:, index])
    mape_losses_by_prediction_variable[str(index)] = calculate_mape_loss(t2, t1).item()
print(mape_losses_by_prediction_variable)

mape_losses = []
for key in mape_losses_by_prediction_variable:
    mape_losses.append(mape_losses_by_prediction_variable[key])
print(mape_losses)

comparison_losses = []
for key in comparison_experiment['evaluation']['mape_losses_by_prediction_variable']:
    comparison_losses.append(comparison_experiment['evaluation']['mape_losses_by_prediction_variable'][key])

plt.plot(mape_losses, label='Ensemble (4x Transformer + 1x Linear Regr.)')
plt.plot(comparison_losses, label='NeuralNet (2 Layers x 2048 Neurons)')
plt.legend()
plt.show()